In [1]:
# https://arxiv.org/pdf/1807.06560v1.pdf?ref=https://githubhelp.com
# https://github.com/renatolfc/chimera-stf/blob/master/chimera/chimera.py

In [2]:
import itertools
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SGD
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import make_blobs

In [15]:
num_communities = 4
num_features = 5
community_sizes = [25] * num_communities

def community_node_feature_params(num_features, num_communities):
    mu = [1, -1] * num_features
    mu = list(set(itertools.permutations(mu, num_features)))[:num_communities]

    sigma_sq = [0.25, 0.5] * num_features
    sigma_sq = list(set(itertools.permutations(sigma_sq, num_features)))[:num_communities]

    return mu, sigma_sq


In [16]:
community_node_feature_params(num_communities, num_features)

([(-1, 1, 1, -1, -1),
  (-1, 1, -1, 1, -1),
  (1, -1, 1, 1, -1),
  (1, -1, -1, -1, -1)],
 [(0.25, 0.25, 0.5, 0.25, 0.5),
  (0.25, 0.25, 0.5, 0.25, 0.25),
  (0.25, 0.25, 0.25, 0.5, 0.5),
  (0.25, 0.25, 0.25, 0.5, 0.25)])

In [212]:
# class BlobDataset(Dataset):
     
#     def __init__(self, num_samples):
        
#         centers = [[-1, 1], [1, 1], [-1, -1], [1, -1]]
#         features, labels = make_blobs(n_samples=num_samples, centers=centers, cluster_std=0.7, random_state=40)
        
#         self.features = torch.from_numpy(features).float()
#         self.labels = torch.from_numpy(labels).long()
     
#     def __len__(self):
#         return len(self.features)
     
#     def __getitem__(self, i):
#         return self.features[i], self.labels[i]
     
#     @property
#     def shape(self):
#         return (None, 2)


def permute_labels(C_pred, C_true):

    # check shape
    # check one hot

    best_acc = 0.
    best_perm = tuple()

    for perm in list(itertools.permutations(range(C_true.shape[-1]))):
        
        C_pred_prem = C_pred[..., perm]
        acc = ((C_pred_prem == C_true) * 1.).mean().item()

        if acc > best_acc:
            
            best_acc = acc
            best_perm = perm

    return C_pred[..., best_perm], best_perm

In [207]:
class KMeans(nn.Module):
    '''k-means clustering using the Lloyd algorithm with Forgy initialization and Euclidean distance'''
     
    def __init__(self, num_feats, num_clusters, epsilon=1e-10, device=torch.device("cpu"), seed=1234):

        super().__init__()

        self.num_feats = num_feats
        self.num_clusters = num_clusters
        self.epsilon = epsilon
        
    
        self.C = torch.zeros(self.num_clusters, self.num_feats).to(device)
        self.C_init = False

        # self.num_samples_in_clusters = torch.ones(self.num_clusters).to(device)

    
    def _initialize_C(self, X):
        
        self.C += X[np.random.choice(len(X), self.num_clusters, replace=False)]
        self.C_init = True
     

    def _euclidean_distance(self, x1, x2):
        
        dist = (x1.unsqueeze(dim=-2) - x2.unsqueeze(dim=-3)).pow(2).sum(dim=-1).squeeze()
    
        return dist.clamp_min_(self.epsilon)


    def _update_C(self, X, cluster):
        
        matched_cluster, _ = cluster.unique(return_counts=True)
        
        C = torch.zeros_like(self.C)
        mask = (cluster[None].expand(len(matched_cluster), -1) == matched_cluster[:, None]).float()
        C[matched_cluster] = mask @ X / mask.sum(dim=-1)[..., :, None]

        self.C = C


    def _assign_cluster(self, X):

        # distance of samples to cluster centroids
        distance = self._euclidean_distance(X, self.C)
        # assign samples to nearest cluster
        cluster = distance.argmin(dim=-1)

        return cluster

    
    def predict(self, X, y=None):

        y_pred = self._assign_cluster(X)
        y_pred = F.one_hot(y_pred, num_classes=self.num_clusters)

        if y is not None:
            y_pred, _ = permute_labels(y_pred , y)

        return y_pred
        

    def forward(self, X):

        if not self.C_init:
            self._initialize_C(X)

        C_init = self.C

        X_cluster = self._assign_cluster(X)
        self._update_C(X, X_cluster)

        loss = (self.C - C_init).pow(2).sum()

        # lr = 1 / self.num_samples_in_clusters[:, None] * 0.9 + 0.1
        # self.num_samples_in_clusters[matched_cluster] += count
        # print(self.num_samples_in_clusters)
        
        # self.C = self.C * (1-lr) + C * lr

        return loss

In [211]:
# num_samples = 6000
# num_clusters = 4
# num_epochs = 200
# batch_size = 1000

# dataset = BlobDataset(num_samples=num_samples)
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# num_feats = dataloader.dataset.shape[-1]
# kmeans = KMeans(num_feats, num_clusters)

# for epoch in range(num_epochs):

#     for X, y in dataloader:

#         loss = kmeans(X)

#     print("epoch {:4d} of {:4d} | loss {:6.4f}".format(epoch, num_epochs, loss.item()))

In [209]:
# y = F.one_hot(dataset.labels, num_classes=num_clusters)
# # distance of samples to cluster centroids
# y_pred = kmeans.predict(dataset.features, y)

# acc = ((y_pred.argmax(-1) == y.argmax(-1)) * 1.).mean().item() * 100

# print(round(acc, 2))

In [ ]:
class NMF(nn.Module):
    '''...'''
     
    def __init__(self, num_samples, num_nodes, num_feats, time_len, latent_dim, beta=1., lambda1=1., lambda2=1., device=torch.device("cpu")):

        super().__init__()

        self.U = nn.Parameter(torch.rand(num_samples, time_len, num_nodes, latent_dim)).to(device)
        self.V = nn.Parameter(torch.rand(num_samples, num_nodes, latent_dim)).to(device)
        self.W = nn.Parameter(torch.rand(num_samples, num_feats, latent_dim)).to(device)

        self.beta = beta
        self.lambda1 = lambda1
        self.lambda2 = lambda2


    def forward(self, A, X, index=None):

        if index is None:
            index = list(range(A.shape[0]))

        U = self.U[index, ...].clamp_min(0.)
        V = self.V[index, None, ...].clamp_min(0.)
        W = self.W[index, None, ...].clamp_min(0.)

        error_A = (A - U.matmul(V.transpose(-2, -1))).pow(2).sum((-3, -2, -1))
        error_X = (X - U.matmul(W.transpose(-2, -1))).pow(2).sum((-3, -2, -1))
        
        norm_V = V.pow(2).sum((-3, -2, -1))
        norm_W = W.pow(2).sum((-3, -2, -1))
        norm_U = U.pow(2).sum((-3, -2, -1))
        norm_U_diff = U.diff(n=1, dim=-3).sum((-3, -2, -1))

        loss = error_A + (self.beta * error_X) + self.lambda1 * (norm_V + norm_W + norm_U) + (self.lambda2 * norm_U_diff)

        return loss


In [217]:
data_path = "../data/data.npz"

data_dic = dict(np.load(data_path, allow_pickle=True).items())
dataset = torch.utils.data.TensorDataset(torch.from_numpy(data_dic["A"]).float())

num_samples, time_len, num_nodes, _ = dataset.tensors[0].shape
num_feats = num_nodes

dataloader = torch.utils.data.DataLoader(dataset, batch_size=num_samples, shuffle=False)

In [215]:
dataset.tensors[0]

torch.Size([20, 10, 100, 100])

In [210]:
class Chimera(nn.Module):
    '''...'''

    def __init__(self, num_samples, num_nodes, num_feats, time_len, num_communities, latent_dim, beta=1., lambda1=1., lambda2=1., device=torch.device("cpu")):

        super().__init__()

        self.num_nodes = num_nodes
        self.num_feats = num_feats
        self.time_len = time_len
        self.latent_dim = latent_dim

        self.nmf = NMF(num_samples, num_nodes, num_feats, time_len, latent_dim, beta, lambda1, lambda2, device)
        self.k_means = KMeans(latent_dim, num_communities, device)
    

In [ ]:
chimera = Chimera()

optimiszer = ()

for X in dataloader:
    X = X.to(device)
    loss = chimera.nmf(X)

node_embeddings = chimera.nmf.U
del chimera.nmf

node_embeddings = node_embeddings.reshape(-1, latent_dim)
dataloader = DataLoader(node_embeddings, batch_size=batch_size, shuffle=True)

for U in dataloader:
    U = U.to(device)
    loss = chimera.k_means(U)



In [36]:
num_samples = 7
time_len = 11
num_nodes = 10
num_feats = 5 
num_communities = 3

A = torch.rand(num_samples, time_len, num_nodes, num_nodes)
X = torch.rand(num_samples, time_len, num_nodes, num_feats)

chimera = Chimera(num_samples, num_nodes, num_feats, time_len, num_communities)

chimera.matrix_factorization_loss(A[[0, 1, 3], ...], X[[0, 1, 3], ...], index=[0, 1, 3])

tensor([554.8316, 582.7816, 623.6739], grad_fn=<AddBackward0>)